In [6]:
from scipy.signal import lfilter, lfilter_zi
import numpy as np
import pyaudio as pa
from scipy.io import wavfile

In [7]:
def tofloat32(x): return (x / np.abs(x).max()).astype(np.float32)

In [8]:
fs, data = wavfile.read('GOT.wav')
y = tofloat32(data)

In [9]:
def comb_filter(fs : int, data, tau : float, T60 : float, zi = None) -> tuple[np.array, np.array]:
    N = int(np.round(tau * fs, 0))
    a = np.zeros(N + 1)
    a[[0, N]] = [1, 10 ** (-3 * tau / T60)]
    b = [1.0]
    
    if zi is None: zi = np.zeros(lfilter_zi(b, a).size)
        
    return lfilter(b, a, data, zi = zi)

In [15]:
p = pa.PyAudio()

chunksize, tau, T60 = 1024, 80e-3, .5

stream = p.open(channels = 1, format = pa.paFloat32, rate = fs, output = True)
stream.start_stream()

y_chunk = y[ : chunksize]
y_filtered, zi = comb_filter(fs, y_chunk, tau, T60)
y_filtered = tofloat32(y_filtered).tobytes()

i = 1
while len(y_filtered) > 0:
    stream.write(y_filtered)
    y_chunk = y[i * chunksize : (i + 1) * chunksize]
    y_filtered, zi = comb_filter(fs, y_chunk, tau, T60, zi)
    if y_filtered.size and np.count_nonzero(y_filtered): y_filtered = tofloat32(y_filtered).tobytes()
    i += 1
    
stream.stop_stream()
stream.close()
p.terminate()

In [16]:
def Karplus_Strong(freq : float, dur : float, fs : float, S : float, b) -> np.array:
    N = int(np.round(fs/freq - 1/(2 * S), 0))
    samples = np.zeros(int(np.round(fs * dur, 0)))

    try: samples[ : N] = (2 * np.random.randint(0, 2, N) - 1).astype(float)
    except ValueError: return samples
    k = np.zeros(N)
    r = np.random.binomial(1, 1 / S, samples.size).astype(bool)

    for i in range(N, N * (1 + samples.size//N), N):
        idx = r[i : i + N]
        k = k[ : idx.size]

        t1 = samples[i - N : i - N + k.size]
        if i == N: t2 = np.concatenate(([samples[i - N - 1]], samples[i - N : i - N - 1 + k.size]))
        else: t2 =  samples[i - N - 1 : i - N - 1 + k.size]

        k[~idx] = t1[~idx]
        k[idx] = (t1 + t2)[idx]/2

        samples[i : i + N] = b * k

    return samples

In [17]:
from IPython.display import Audio

In [36]:
fs = 8000
freq = 40
b = 1
dur = 1.5
S = 1
sound = Karplus_Strong(freq = freq, fs = fs, b = b, dur = dur, S = 1)

Audio(sound, rate = fs)